# Loading and Displaying Data

In [ ]:
import pandas as pd

# Load the datasets
real_news = pd.read_csv('True.csv')

# Printing head and tail
print(real_news)

# Print length => rows
print("Length (rows):", len(real_news))


                                                   title  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
21412  'Fully committed' NATO backs new U.S. approach...   
21413  LexisNexis withdrew two products from Chinese ...   
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1      WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2      WASHINGTON (Reuters) - The special counsel inv... 

In [ ]:
import pandas as pd

# Load the datasets
fake_news = pd.read_csv('Fake.csv')

# Printing head and tail
print(fake_news)

# Print length => rows
print("Length (rows):", len(fake_news))


                                                   title  \
0       Donald Trump Sends Out Embarrassing New Year’...   
1       Drunk Bragging Trump Staffer Started Russian ...   
2       Sheriff David Clarke Becomes An Internet Joke...   
3       Trump Is So Obsessed He Even Has Obama’s Name...   
4       Pope Francis Just Called Out Donald Trump Dur...   
...                                                  ...   
23476  McPain: John McCain Furious That Iran Treated ...   
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text      subject  \
0      Donald Trump just couldn t wish all Americans ...         News   
1      House Intelligence Committee Chairman Devin Nu...         News   
2      On Friday, it was revealed that former Milwauk...    

# Combining and Shuffling the data

In [ ]:
# Load the dataset
fake_news = pd.read_csv('Fake.csv')
real_news = pd.read_csv('True.csv')

# Combine the datasets and shuffle
fake_news['label'] = 0    # adds a column
real_news['label'] = 1

data = pd.concat([fake_news, real_news]).sample(frac=1.0)   # frac 1.0 => returns all the data (rows)

# Write the combined data to a new CSV file
data.to_csv('Combined.csv', index=False)

print("Data loaded successfully and combined CSV generated!")

data

Data loaded successfully and combined CSV generated!


,title,text,subject,date,label
2975,Lawmakers unveil bill to keep insurance expert...,WASHINGTON (Reuters) - The U.S. Senate Banking...,politicsNews,"June 28, 2017",1
2883,GOP Lawmaker Deactivates Twitter Account Afte...,A Nebraska State Senator was busted previously...,News,"January 23, 2017",0
4011,BUSTED: Trump Supporter Tries To Rig Election...,Donald Trump has repeatedly accused Democrats ...,News,"October 28, 2016",0
17737,BREAKING: ONLY MLB Player To Kneel During Nati...,When Oakland Athletics catcher Bruce Maxwell t...,left-news,"Oct 29, 2017",0
20709,OBAMA’S ECONOMIC LEGACY IN 9 Easy To Read Charts,Anyone who would look at these charts and stil...,left-news,"Apr 19, 2016",0
...,...,...,...,...,...
15020,"Last exit from Myanmar, Rohingya wait for week...","AH LEI THAN KYAW, Myanmar (Reuters) - Some 1,0...",worldnews,"November 12, 2017",1
1494,Wealthy financiers could gain from Trump's pro...,WASHINGTON (Reuters) - High-income Wall Street...,politicsNews,"September 28, 2017",1
5710,"Tillerson calls counterparts from Mexico, Cana...",WASHINGTON (Reuters) - U.S. Secretary of State...,politicsNews,"February 2, 2017",1
6963,Trump takes populist message to U.S. heartland...,CINCINNATI (Reuters) - President-elect Donald ...,politicsNews,"December 1, 2016",1


# Preprocess Function

In [ ]:
import re
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def preprocess_text(text):
    # Replace characters that are not between a to z or A to Z with whitespace
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert all characters into lowercase
    text = text.lower()

    # Remove inflectional morphemes like "ed", "est", "s", and "ing" from their token stem
    text = [stemmer.stem(word) for word in text.split()]

    # Join the processed words back into a single string
    text = ' '.join(text)

    return text


# Training the Model


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Check if GPU is available
if tf.test.is_gpu_available():
    device_name = tf.test.gpu_device_name()
else:
    device_name = 'CPU:0'
print('Using device:', device_name)

# Load the dataset
data = pd.read_csv('Combined.csv')

# Preprocess the text using the preprocessing function
print("\nPreprocessing data...")

data['title_preprocessed'] = data['title'].apply(preprocess_text)

# Split the dataset into train, validation, and test sets
train_ratio = 0.64
val_ratio = 0.16
test_ratio = 0.2

print("\nSplitting Data...")

train_data = data.sample(frac=train_ratio, random_state=42)
remaining_data = data.drop(train_data.index)

val_data = remaining_data.sample(frac=val_ratio/(val_ratio+test_ratio), random_state=42)
test_data = remaining_data.drop(val_data.index)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the titles and convert them into BERT input tensors
print("\nTokenizing and Converting...")

train_inputs = tokenizer(list(train_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')
val_inputs = tokenizer(list(val_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')
test_inputs = tokenizer(list(test_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')

# Convert the labels into TensorFlow tensors
train_labels = tf.convert_to_tensor(list(train_data['label']))
val_labels = tf.convert_to_tensor(list(val_data['label']))
test_labels = tf.convert_to_tensor(list(test_data['label']))

# Extract token tensors, segment tensors, and mask tensors from the BERT inputs
train_token_tensors = train_inputs['input_ids']
train_segment_tensors = train_inputs['token_type_ids']
train_mask_tensors = train_inputs['attention_mask']

val_token_tensors = val_inputs['input_ids']
val_segment_tensors = val_inputs['token_type_ids']
val_mask_tensors = val_inputs['attention_mask']

test_token_tensors = test_inputs['input_ids']
test_segment_tensors = test_inputs['token_type_ids']
test_mask_tensors = test_inputs['attention_mask']

# Build the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
print("\nCompiling Model...")

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the BERT model
print("\nTraining Model...")

batch_size = 64
num_epochs = 4

history = model.fit([train_token_tensors, train_segment_tensors, train_mask_tensors], train_labels, batch_size=batch_size, epochs=num_epochs, validation_data=([val_token_tensors, val_segment_tensors, val_mask_tensors], val_labels))

# Save the trained model
print("\nSaving Model...")

model.save_pretrained('/content/bertv3_model')



Using device: /device:GPU:0

Preprocessing data...

Splitting Data...

Tokenizing and Converting...


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Compiling Model...

Training Model...
Epoch 1/4
449/449 [==============================] - 324s 586ms/step - loss: 0.1729 - accuracy: 0.9346 - val_loss: 0.1158 - val_accuracy: 0.9613
Epoch 2/4
449/449 [==============================] - 265s 590ms/step - loss: 0.0713 - accuracy: 0.9749 - val_loss: 0.0845 - val_accuracy: 0.9697
Epoch 3/4
449/449 [==============================] - 267s 595ms/step - loss: 0.0421 - accuracy: 0.9857 - val_loss: 0.1029 - val_accuracy: 0.9691
Epoch 4/4
449/449 [==============================] - 264s 589ms/step - loss: 0.0226 - accuracy: 0.9922 - val_loss: 0.1218 - val_accuracy: 0.9681

Saving Model...


# Validation

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate([test_token_tensors, test_segment_tensors, test_mask_tensors], test_labels, batch_size=batch_size)
print(f'Test loss: {loss * 100:.3f}%')
print(f'Test accuracy: {accuracy * 100:.3f}%')



141/141 [==============================] - 26s 186ms/step - loss: 0.1080 - accuracy: 0.9700
Test loss: 10.803%
Test accuracy: 97.004%


# Using the Model to predict

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load the saved model
model = TFBertForSequenceClassification.from_pretrained('/content/bertv3_model')

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example input text
input_text = input("\n\nEnter News Title: ")

# Preprocess the input text
preprocessed_text = preprocess_text(input_text)

print("Preprocessed Text:", preprocessed_text)

# Tokenize the preprocessed text
inputs = tokenizer(preprocessed_text, truncation=True, padding='max_length', max_length=42, return_tensors='tf')

# Extract input tensors
token_tensors = inputs['input_ids']
segment_tensors = inputs['token_type_ids']
mask_tensors = inputs['attention_mask']

# Make predictions
predictions = model.predict([token_tensors, segment_tensors, mask_tensors])
logits = predictions.logits[0]
probabilities = tf.nn.softmax(logits)
predicted_label = tf.argmax(probabilities)

# Print the predicted label and probabilities
if predicted_label == 0:
    print("\n*-*-Fake News-*-*")
else:
    print("\n*-*-Real News-*-*")

print("\nProbability of being fake: {:.2%}".format(probabilities[0]))
print("Probability of being real: {:.2%}".format(probabilities[1]))


Some layers from the model checkpoint at /content/bertv3_model were not used when initializing TFBertForSequenceClassification: ['dropout_149']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/bertv3_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.




Enter News Title: USA is bankrupted
Preprocessed Text: usa is bankrupt
1/1 [==============================] - 4s 4s/step

Fake news

Probability of being fake: 76.43%
Probability of being real: 23.57%


BREAKING: MUSLIM OPENS FIRE ON JOURNALISTS [Video]

 WATCH: Wolf Blitzer Makes Republican Throw Temper Tantrum Over Trumpâ€™s Nazi Problem

 Boston men jailed for Trump-inspired hate crime attack



In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.1 MB/s eta 0:00:00
